# Graficos

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy import stats
from scipy.stats import linregress
import csv
import os

In [ ]:
dataframe = pd.read_csv('output.csv', sep=';')

In [ ]:
tiempos_promedio_nx = []
tiempos_promedio_ht = []
tiempos_promedio_str = []
valor_inicial = 100

for i in range(16):
    tiempos_promedio_nx.append(float(dataframe.iloc[valor_inicial, 1])/100)
    tiempos_promedio_ht.append(float(dataframe.iloc[valor_inicial, 3])/100)
    tiempos_promedio_str.append(float(dataframe.iloc[valor_inicial, 5])/100)
    valor_inicial += 102


etiquetas_x = [f'2^{i}' for i in range(10, 26)]

plt.figure(figsize=(10, 6))
plt.plot(etiquetas_x, tiempos_promedio_nx, marker='o',
         color='b', linestyle='-', label='Método 1: Nearest X')
plt.plot(etiquetas_x, tiempos_promedio_ht, marker='o', color='r',
         linestyle=':', label='Método 2: Hilbert R-tree')
plt.plot(etiquetas_x, tiempos_promedio_str, marker='o',
         color='k', linestyle='--', label='Método 3: STR')
plt.legend()
plt.xlabel('Tamaño de entrada de los datos')
plt.ylabel('Tiempo de búsqueda promedio (ns)')
plt.title(
    'Tiempos de búsqueda promedio de rectángulos en función del tamaño de entrada')
plt.xticks(rotation=45)
plt.tight_layout()


plt.savefig('resultados/grafico_tiempos_promedio.pdf')
plt.show()

In [ ]:
ios_promedio_nx = []
ios_promedio_ht = []
ios_promedio_str = []
valor_columna = 100

for i in range(16):
    ios_promedio_nx.append(int(dataframe.iloc[valor_columna, 2])/100)
    ios_promedio_ht.append(int(dataframe.iloc[valor_columna, 4])/100)
    ios_promedio_str.append(int(dataframe.iloc[valor_columna, 6])/100)
    valor_columna += 102

etiquetas_x = [f'2^{i}' for i in range(10, 26)]

plt.figure(figsize=(10, 6))
plt.plot(etiquetas_x, ios_promedio_nx, marker='o', color='b', linestyle='-', label='Método 1: Nearest X')
plt.plot(etiquetas_x, ios_promedio_ht, marker='o', color='r', linestyle=':', label='Método 2: Hilbert R-tree')
plt.plot(etiquetas_x, ios_promedio_str, marker='o', color='k', linestyle='--', label='Método 3: STR')
plt.legend()
plt.xlabel('Tamaño de entrada de los datos')
plt.ylabel('I/Os promedio de la consulta')
plt.title('Cantidad de accesos promedio a bloques de disco de la búsqueda en función del tamaño de entrada')


plt.xticks(rotation=45)
plt.tight_layout()


plt.savefig('resultados/grafico_ios.pdf')
plt.show()

In [ ]:
# Generador de csv para generar estadísticas de los resultados de los algoritmos de cada n.

filas_por_archivo = 102
subconjunto = -1
contador = 0

while contador < 16:
    inicio = contador * filas_por_archivo
    fin = (contador + 1) * filas_por_archivo

    subconjunto = dataframe.iloc[inicio:fin]

    nombre_archivo = f'resultados/n_{contador + 10}.csv'
    subconjunto.to_csv(nombre_archivo, sep=';', index=False, header=" ")
    contador += 1

directorio = 'resultados'

for archivo in os.listdir(directorio):
    if archivo.endswith('.csv'):
        with open(os.path.join(directorio, archivo), 'r') as file:
            lineas = file.readlines()
        if archivo == "n_25.csv":
            with open(os.path.join(directorio, archivo), 'w') as file:
                file.writelines(lineas)
        else:
            lineas = lineas[:-1]
            with open(os.path.join(directorio, archivo), 'w') as file:
                file.writelines(lineas)

for archivo in os.listdir(directorio):
    if archivo.endswith('.csv'):
        with open(os.path.join(directorio, archivo), 'r') as file:
            lineas = file.readlines()
            lineas = lineas[:-1]
            with open(os.path.join(directorio, archivo), 'w') as file:
                file.writelines(lineas)

In [ ]:
def bootstrap_ci(data, n_bootstrap, alpha=0.05, random_seed=123):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, len(data), replace=True)
        bootstrapped_means.append(np.mean(sample))

    lower_percentile = 100 * alpha / 2
    upper_percentile = 100 * (1 - alpha / 2)

    lower_bound = np.percentile(bootstrapped_means, lower_percentile)
    upper_bound = np.percentile(bootstrapped_means, upper_percentile)

    return lower_bound, upper_bound

In [ ]:
print(r"\begin{table}[H]")
print(r"\begin{center}")
print(r"\begin{tabular}{|c|c|c|c|c|c|c|}")
print(r"\hline")
print(r"Input & media de tiempo (ns) & d.e tiempo (ns) & I.C de tiempo (ns) & I.C de I/Os & d.e I/Os \\")
print(r"\hline")

for i in range(10, 26):
    archivo_csv = f'resultados/n_{i}.csv'
    df = pd.read_csv(archivo_csv, sep=';')
    
    columna_tiempo = df[' Tiempo NearestX ']
    columna_IOs = df[' IOs NearestX ']
    
    n_bootstrap = 1000
    lower_bound_tiempo, upper_bound_tiempo = bootstrap_ci(columna_tiempo, n_bootstrap)
    lower_bound_IOs, upper_bound_IOs = bootstrap_ci(columna_IOs, n_bootstrap)

    print(f"$2^{{{i}}}$ & {round(columna_tiempo.mean(), 2)} & {round(columna_tiempo.std(), 2)} & ({round(lower_bound_tiempo, 2)}, {round(upper_bound_tiempo, 2)}) & ({round(lower_bound_IOs, 2)}, {round(upper_bound_IOs, 2)}) & {round(columna_IOs.std(), 2)} \\\\")
    
print(r"\hline")
print(r"\end{tabular}")
print(r"\caption{}")
print(r"\end{center}")
print(r"\end{table}")

In [ ]:
print(r"\begin{table}[H]")
print(r"\begin{center}")
print(r"\begin{tabular}{|c|c|c|c|c|c|c|}")
print(r"\hline")
print(r"Input & media de tiempo (ns) & d.e tiempo (ns) & I.C de tiempo (ns) & I.C de I/Os & d.e I/Os \\")
print(r"\hline")

for i in range(10, 26):
    archivo_csv = f'resultados/n_{i}.csv'
    df = pd.read_csv(archivo_csv, sep=';')
    
    columna_tiempo = df[' Tiempo Hilbert']
    columna_IOs = df[' IOs Hilbert']
    
    n_bootstrap = 1000
    lower_bound_tiempo, upper_bound_tiempo = bootstrap_ci(columna_tiempo, n_bootstrap)
    lower_bound_IOs, upper_bound_IOs  = bootstrap_ci(columna_IOs, n_bootstrap)

    print(f"$2^{{{i}}}$ & {round(columna_tiempo.mean(), 2)} & {round(columna_tiempo.std(), 2)} & ({round(lower_bound_tiempo, 2)}, {round(upper_bound_tiempo, 2)}) & ({round(lower_bound_IOs, 2)}, {round(upper_bound_IOs, 2)}) & {round(columna_IOs.std(), 2)} \\\\")
    
print(r"\hline")
print(r"\end{tabular}")
print(r"\caption{}")
print(r"\end{center}")
print(r"\end{table}")

In [ ]:
print(r"\begin{table}[H]")
print(r"\begin{center}")
print(r"\begin{tabular}{|c|c|c|c|c|c|c|}")
print(r"\hline")
print(r"Input & media de tiempo (ns) & d.e tiempo (ns) & I.C de tiempo (ns) & I.C de I/Os & d.e I/Os \\")
print(r"\hline")

for i in range(10, 26):
    archivo_csv = f'resultados/n_{i}.csv'
    df = pd.read_csv(archivo_csv, sep=';')
    
    columna_tiempo = df[' Tiempo STR']
    columna_IOs = df[' IOS STR']
    
    n_bootstrap = 1000
    lower_bound_tiempo, upper_bound_tiempo = bootstrap_ci(columna_tiempo, n_bootstrap)
    lower_bound_IOs, upper_bound_IOs = bootstrap_ci(columna_IOs, n_bootstrap)

    print(f"$2^{{{i}}}$ & {round(columna_tiempo.mean(), 2)} & {round(columna_tiempo.std(), 2)} & ({round(lower_bound_tiempo, 2)}, {round(upper_bound_tiempo, 2)}) & ({round(lower_bound_IOs, 2)}, {round(upper_bound_IOs, 2)}) & {round(columna_IOs.std(), 2)} \\\\")
    
print(r"\hline")
print(r"\end{tabular}")
print(r"\caption{}")
print(r"\end{center}")
print(r"\end{table}")

In [ ]:
tiempos_promedio_nx = []
valor_inicial = 100

for i in range(11):
    tiempos_promedio_nx.append(float(dataframe.iloc[valor_inicial, 1])/100)
    tiempos_promedio_ht.append(float(dataframe.iloc[valor_inicial, 3])/100)
    tiempos_promedio_str.append(float(dataframe.iloc[valor_inicial, 5])/100)
    valor_inicial += 102

etiquetas_x = [f'2^{i}' for i in range(10, 21)]

x = np.array([2**i for i in range(10, 21)])
y = np.array(tiempos_promedio_nx)

log_x = np.log2(x)
log_y = np.log(y)

slope, intercept, r_value, p_value, std_err = linregress(log_x, log_y)

a = slope
c = np.exp(intercept)

plt.figure(figsize=(10, 6))
plt.plot(etiquetas_x, tiempos_promedio_nx, marker='o', color='b', linestyle='-', label='Método 1: Nearest X')
plt.plot(etiquetas_x, y_estimado, marker='o', color='g', linestyle='--', label= f"Curva estimada: c = {round(c, 4)}, α = {round(a, 4)}")
plt.legend()
plt.xlabel('Tamaño de entrada de los datos')
plt.ylabel('Tiempo de búsqueda promedio (ns)')
plt.title(
    'Tiempos de búsqueda promedio de rectángulos en función del tamaño de entrada')
plt.xticks(rotation=45)
plt.tight_layout()


plt.savefig('resultados/grafico_curva_nx.pdf')
plt.show()

In [ ]:
tiempos_promedio_ht = []
valor_inicial = 100

for i in range(11):
    tiempos_promedio_nx.append(float(dataframe.iloc[valor_inicial, 1])/100)
    tiempos_promedio_ht.append(float(dataframe.iloc[valor_inicial, 3])/100)
    tiempos_promedio_str.append(float(dataframe.iloc[valor_inicial, 5])/100)
    valor_inicial += 102

etiquetas_x = [f'2^{i}' for i in range(10, 21)]

x = np.array([2**i for i in range(10, 21)])
y = np.array(tiempos_promedio_ht)

log_x = np.log2(x)
log_y = np.log(y)

slope, intercept, r_value, p_value, std_err = linregress(log_x, log_y)

a = slope
c = np.exp(intercept)

plt.figure(figsize=(10, 6))
plt.plot(etiquetas_x, tiempos_promedio_ht, marker='o', color='r', linestyle='-', label='Método 2: Hilbert R-tree')
plt.plot(etiquetas_x, y_estimado, marker='o', color='m', linestyle='--', label= f"Curva estimada: c = {round(c, 4)}, α = {round(a, 4)}")
plt.legend()
plt.xlabel('Tamaño de entrada de los datos')
plt.ylabel('Tiempo de búsqueda promedio (ns)')
plt.title(
    'Tiempos de búsqueda promedio de rectángulos en función del tamaño de entrada')
plt.xticks(rotation=45)
plt.tight_layout()


plt.savefig('resultados/grafico_curva_ht.pdf')
plt.show()

In [ ]:
tiempos_promedio_str = []
valor_inicial = 100

for i in range(11):
    tiempos_promedio_nx.append(float(dataframe.iloc[valor_inicial, 1])/100)
    tiempos_promedio_ht.append(float(dataframe.iloc[valor_inicial, 3])/100)
    tiempos_promedio_str.append(float(dataframe.iloc[valor_inicial, 5])/100)
    valor_inicial += 102

etiquetas_x = [f'2^{i}' for i in range(10, 21)]

x = np.array([2**i for i in range(10, 21)])
y = np.array(tiempos_promedio_str)

log_x = np.log2(x)
log_y = np.log(y)

slope, intercept, r_value, p_value, std_err = linregress(log_x, log_y)

a = slope
c = np.exp(intercept)

plt.figure(figsize=(10, 6))
plt.plot(etiquetas_x, tiempos_promedio_str, marker='o', color='k', linestyle='-', label='Método 3: STR')
plt.plot(etiquetas_x, y_estimado, marker='o', color='c', linestyle='--', label= f"Curva estimada: c = {round(c, 4)}, α = {round(a, 4)}")
plt.legend()
plt.xlabel('Tamaño de entrada de los datos')
plt.ylabel('Tiempo de búsqueda promedio (ns)')
plt.title(
    'Tiempos de búsqueda promedio de rectángulos en función del tamaño de entrada')
plt.xticks(rotation=45)
plt.tight_layout()


plt.savefig('resultados/grafico_curva_str.pdf')
plt.show()